# JsonOutputParser

JsonOutputParser는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구입니다. 이 도구는 Large Language Model (LLM)이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계되었습니다.

LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량(예: 인텔리전스)이 충분히 커야 합니다. 예를 들어, llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리합니다.

**[참고]**

`JSON (JavaScript Object Notation)` 은 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 교환 포맷입니다. 웹 개발에서 매우 중요한 역할을 하며, 서버와 클라이언트 간의 통신을 위해 널리 사용됩니다. JSON은 읽기 쉽고, 기계가 파싱하고 생성하기 쉬운 텍스트를 기반으로 합니다.

JSON의 기본 구조
JSON 데이터는 이름(키)과 값의 쌍으로 이루어져 있습니다. 여기서 "이름"은 문자열이고, "값"은 다양한 데이터 유형일 수 있습니다. JSON은 두 가지 기본 구조를 가집니다:

- 객체: 중괄호 {}로 둘러싸인 키-값 쌍의 집합입니다. 각 키는 콜론 :을 사용하여 해당하는 값과 연결되며, 여러 키-값 쌍은 쉼표 ,로 구분됩니다.
- 배열: 대괄호 []로 둘러싸인 값의 순서 있는 목록입니다. 배열 내의 값은 쉼표 ,로 구분됩니다.

```json
{
  "name": "John Doe",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "123 Main St",
    "city": "Anytown"
  }
}
```

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [3]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [5]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간단한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상")

In [7]:
# 질문 
question = "프로그래밍 언어에 대해 설명"

# # 파서 설정
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\ub2e8\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 친절한 AI 어시스턴스 입니다. 질문에 간결하게 답변하세요"),
        ("user","#Format : {format_instruction}\n\n #Question:{question}"),
    ]
)

prompt = prompt.partial(format_instruction=parser.get_format_instructions())

chain = prompt | model | parser

chain.invoke({"question":question})

{'description': '프로그래밍 언어는 컴퓨터 프로그램을 작성하기 위해 사용되는 언어입니다.',
 'hashtags': '#프로그래밍 #언어'}

# Pydantic 없이 사용

In [9]:
# 질의 생성
question = "지구 온난화에 대해 알려주세요. 설명은 `description` 에 관련 키워드는 `hashtags`에 담아주세요."

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 친절한 AI 어시스턴스 입니다. 질문에 간결하게 답변하세요"),
        ("user","#Format : {format_instruction}\n\n #Question:{question}"),
    ]
)

prompt = prompt.partial(format_instruction=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"question":question})

print(response)

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가로 인해 발생합니다. 이로 인해 기후 변화, 해수면 상승, 생태계 파괴 등의 문제가 발생하고 있습니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#해수면상승', '#생태계파괴']}
